## **Python Data Analysis Automation**

In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, Border, Side
from openpyxl.chart import BarChart, Reference
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

def generate_flexible_report(
    file_path,
    group_by_cols,
    value_col,
    output_file='Flexible_Report.xlsx',
    top_n=10
):
    # 1. Load dataset
    df = pd.read_csv(file_path, encoding='ISO-8859-1')

    # 2. Check if all columns exist
    missing = [col for col in group_by_cols + [value_col] if col not in df.columns]
    if missing:
        raise ValueError(f"These columns are missing from the dataset: {missing}")

    # 3. Group and summarize
    grouped = df.groupby(group_by_cols)[value_col].sum().reset_index()
    grouped = grouped.sort_values(by=value_col, ascending=False).head(top_n).round(2)

    # 4. Export to Excel (keep all columns separate!)
    grouped.to_excel(output_file, sheet_name='report', index=False, startrow=3)

    # 5. Load Excel workbook
    wb = load_workbook(output_file)
    ws = wb['report']

    # 6. Chart
    min_row, max_row = ws.min_row, ws.max_row
    num_cols = ws.max_column

    chart = BarChart()
    chart.title = f'Top {top_n} by {value_col}'
    chart.height = 10
    chart.width = 24

    data = Reference(ws, min_col=num_cols, max_col=num_cols, min_row=4, max_row=max_row)
    categories = Reference(ws, min_col=1, max_col=1, min_row=5, max_row=max_row)  # Just first group-by col

    chart.add_data(data, titles_from_data=True)
    chart.set_categories(categories)
    ws.add_chart(chart, "E4")

    # 7. Column widths
    dim_holder = DimensionHolder(worksheet=ws)
    for col in range(1, num_cols + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=30)
    ws.column_dimensions = dim_holder

    # 8. Title & Subtitle
    ws['A1'] = f"{value_col.capitalize()} Report by {', '.join(group_by_cols).capitalize()}"
    ws['A2'] = 'Generated by Python BI Automation'
    ws['A1'].font = Font(size=22, underline='single', color='778899', bold=True)
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=num_cols)

    # 9. Format numeric values
    for row in ws.iter_rows(min_row=5, max_row=max_row, min_col=num_cols, max_col=num_cols):
        for cell in row:
            cell.number_format = '#,##0.00'

    # 10. Add total row
    ws[f'A{max_row + 1}'] = 'Total'
    ws[f'A{max_row + 1}'].font = Font(bold=True)
    ws[f'A{max_row + 1}'].alignment = Alignment(horizontal='center')

    ws[f'{get_column_letter(num_cols)}{max_row + 1}'] = f'=SUM({get_column_letter(num_cols)}5:{get_column_letter(num_cols)}{max_row})'
    ws[f'{get_column_letter(num_cols)}{max_row + 1}'].font = Font(bold=True)
    ws[f'{get_column_letter(num_cols)}{max_row + 1}'].number_format = '#,##0.00'
    ws[f'{get_column_letter(num_cols)}{max_row + 1}'].border = Border(
        top=Side(style='thin'), left=Side(style='thin'),
        right=Side(style='thin'), bottom=Side(style='thin')
    )

    wb.save(output_file)
    print(f"✅ Report saved to '{output_file}'.")

# Example
generate_flexible_report(
    file_path='/content/DataScience_salaries_2025.csv',
    group_by_cols=['job_title', 'company_location'],
    value_col='salary',
    output_file='Salary_By_Job_and_Company.xlsx',
    top_n=10
)

✅ Report saved to 'Salary_By_Job_and_Company.xlsx'.
